# Project: Improving Disaster Resilience

## NOAA Consolidated Historical Events - Extract US Standard Counties and Assign FIPS Code

### 
### Disaster Resilience Project

### Data Used: Standard US Counties FIPS Codes, NOAA Storm Events Details from 1996 to 2022

Andrew Sommers

### Purpose

The notebook will assign standard FIPS codes to the NOAA event data.
NOAA uses US Counties and 'special event zones' to report weather and disaster events.
The notebook will extract only US County data from the NOAA event data and assign the official FIPS code to the county.
Speicial event zones data rows will be skipped and excluded from the output.


#### History 🗓️

Date | Person | Details
---- | ------ | -------
06/01/2023| Andrew Sommers|   Create initial notebook
08/21/2023| Andrew Sommers|   Update the notebook documentation


### Importing Libraries

Import required libraries

In [5]:
#import requests # request http, api
import pandas as pd # tabluar data
#from functools import reduce
import numpy as np
import os
#
pd.options.display.max_columns = None # show all columns in display
pd.options.display.max_rows = None # show all rows in display

In [6]:
# change the notebooks data source directory to the Disaster Resilience  Directory - change this for your local environment
# set the sourcing directory where the 'data folder is located; this is the raw data input files.  
# this notebook assumes data files are located in a 'Data' folder in the following path:
os.chdir('C:\\Users\\andre\\OneDrive\\Documents\\IndianaUniversity\\D592\\Project_Disaster_Resilience\\Data\\Tables')
#
# load the standard FIPS codes and convert to a python dictionary
Standard_FIPS_Input = pd.read_csv(f'FIPS_Prepared_Table.csv', dtype = str)
Standard_FIPS_Codes = Standard_FIPS_Input.copy()
Standard_FIPS_Codes['COUNTY_KEY'] = Standard_FIPS_Codes['STATE_FIPS'] + Standard_FIPS_Codes['COUNTY']  #create a key to match NOAA state and county data to the FIPS table
Standard_FIPS_Codes.drop(["STATE_FIPS", "STATE_CODE", "STATE", "COUNTY","COUNTY_NS", "COUNTY_FULL_NAME", "FIPS_CLASS_CODE", "FUNCTION_STATUS"], axis = 1, inplace=True)
Standard_FIPS_Codes.head(3)

,COUNTY_FIPS,FIPS,COUNTY_KEY
0,1,0500000US01001,1AUTAUGA
1,3,0500000US01003,1BALDWIN
2,5,0500000US01005,1BARBOUR


In [7]:
#create a python dictionary from the FIPS standard codes
FIPS_Dict = dict([(i,[x, y]) for i, x, y in zip(Standard_FIPS_Codes.COUNTY_KEY, Standard_FIPS_Codes.COUNTY_FIPS, Standard_FIPS_Codes.FIPS)])
FIPS_Dict

{'1AUTAUGA': ['1', '0500000US01001'],
 '1BALDWIN': ['3', '0500000US01003'],
 '1BARBOUR': ['5', '0500000US01005'],
 '1BIBB': ['7', '0500000US01007'],
 '1BLOUNT': ['9', '0500000US01009'],
 '1BULLOCK': ['11', '0500000US01011'],
 '1BUTLER': ['13', '0500000US01013'],
 '1CALHOUN': ['15', '0500000US01015'],
 '1CHAMBERS': ['17', '0500000US01017'],
 '1CHEROKEE': ['19', '0500000US01019'],
 '1CHILTON': ['21', '0500000US01021'],
 '1CHOCTAW': ['23', '0500000US01023'],
 '1CLARKE': ['25', '0500000US01025'],
 '1CLAY': ['27', '0500000US01027'],
 '1CLEBURNE': ['29', '0500000US01029'],
 '1COFFEE': ['31', '0500000US01031'],
 '1COLBERT': ['33', '0500000US01033'],
 '1CONECUH': ['35', '0500000US01035'],
 '1COOSA': ['37', '0500000US01037'],
 '1COVINGTON': ['39', '0500000US01039'],
 '1CRENSHAW': ['41', '0500000US01041'],
 '1CULLMAN': ['43', '0500000US01043'],
 '1DALE': ['45', '0500000US01045'],
 '1DALLAS': ['47', '0500000US01047'],
 '1DEKALB': ['49', '0500000US01049'],
 '1ELMORE': ['51', '0500000US01051'],
 '1

In [8]:
# change the notebooks data source directory to the Disaster Resilience  Directory - change this for your local environment
# set the sourcing directory where the 'data folder is located; this is the raw data input files.  
# this notebook assumes data files are located in a 'Data' folder in the following path:
os.chdir('C:\\Users\\andre\\OneDrive\\Documents\\IndianaUniversity\\D592\\Project_Disaster_Resilience\\Data\\NOAA')
#
NOAA_History_Read = pd.read_csv(f'NOAA_EventHistory_1996_2022.csv', dtype = str)
print('Number of Records Loaded = ', len(NOAA_History_Read))

Number of Records Loaded =  1561601


In [9]:
NOAA_History_Read.dropna(inplace = True)
NOAA_History_Input = NOAA_History_Read.copy()
#NOAA_History_Input = NOAA_History_Read[(NOAA_History_Read['STATE'].str.contains("CALIFORNIA"))].copy()# & (df_NOAA['fipsStateCode'] == 8)].copy()
#output the combined data to a temporary csv file
#NOAA_History_Input.to_csv(f'Delete_It.csv', header=True, index=False)
#print('end of file save function - record count of ', len(NOAA_History_Input))
NOAA_History_Input.head(3)

,YEAR,EPISODE_ID,EVENT_ID,EVENT_TYPE,EPISODE_NARRATIVE,STATE,STATE_FIPS,CZ_NAME,CZ_FIPS,CZ_TYPE,DAMAGE_PROPERTY,DAMAGE_CROPS,CUSTOM_EVENT_CODE
0,1996,2052149,5570068,High Wind,Strong gradient winds followed behind a line o...,ILLINOIS,17,HANCOCK,34,Z,0.0,0.0,199610292052149
1,1996,2052748,5570362,Flash Flood,Flash flood waters washed out a small bridge a...,OKLAHOMA,40,BRYAN,13,C,0.0,0.0,199610212052748
2,1996,2052749,5570363,Hail,Hail as large as softballs broke out car and t...,OKLAHOMA,40,STEPHENS,137,C,0.0,0.0,199610202052749


In [10]:
#Replace rows containing actual US Counties with the correct county FIPS code
#Remove any rows that are not actual US Counties
NOAA_History_Input['COUNTY_KEY'] = NOAA_History_Input['STATE_FIPS'] + NOAA_History_Input['CZ_NAME']  #create a common key to align the NOAA data to the FIPS Dictionary
NOAA_History_Input["COUNTY_FIPS"] = NOAA_History_Input["COUNTY_KEY"].map(FIPS_Dict).str[0] #map the standard county FIPS code
NOAA_History_Input["FIPS"] = NOAA_History_Input["COUNTY_KEY"].map(FIPS_Dict).str[1] #map the standard full FIPS code
NOAA_History_Input["COUNTY_FIPS"] = pd.to_numeric(NOAA_History_Input["COUNTY_FIPS"], errors='coerce') #change FIPS codes to numeric but missing values coerced to NaN
NOAA_History_Input = NOAA_History_Input.dropna().reset_index(drop=True) #drop NaN values; rows with no match to the FIPS dictionary
NOAA_History_Input = NOAA_History_Input.astype({"COUNTY_FIPS" : "int"}) #covert the County Key FIPS codes to integers
NOAA_History_Input = NOAA_History_Input.astype({"COUNTY_FIPS" : "string"}) #convert the County Key FIPS codes back to strings
NOAA_History_Input.drop(["CZ_FIPS", "CZ_TYPE", "COUNTY_KEY"], axis = 1, inplace=True)
NOAA_History_Input.rename(columns={'CZ_NAME': 'COUNTY'}, inplace = True)
print('Number of Records Processed = ', len(NOAA_History_Input))

Number of Records Processed =  1062498


In [11]:
NOAA_History_Input.head(5)

,YEAR,EPISODE_ID,EVENT_ID,EVENT_TYPE,EPISODE_NARRATIVE,STATE,STATE_FIPS,COUNTY,DAMAGE_PROPERTY,DAMAGE_CROPS,CUSTOM_EVENT_CODE,COUNTY_FIPS,FIPS
0,1996,2052149,5570068,High Wind,Strong gradient winds followed behind a line o...,ILLINOIS,17,HANCOCK,0.0,0.0,199610292052149,67,0500000US17067
1,1996,2052748,5570362,Flash Flood,Flash flood waters washed out a small bridge a...,OKLAHOMA,40,BRYAN,0.0,0.0,199610212052748,13,0500000US40013
2,1996,2052749,5570363,Hail,Hail as large as softballs broke out car and t...,OKLAHOMA,40,STEPHENS,0.0,0.0,199610202052749,137,0500000US40137
3,1996,2404601,5538413,Blizzard,A major winter storm brought blizzard conditio...,MINNESOTA,27,YELLOW MEDICINE,0.0,0.0,199601172404601,173,0500000US27173
4,1996,2404529,5535851,Thunderstorm Wind,"The ""Best for Less"" hotel office roof was blow...",TEXAS,48,MCLENNAN,25000.0,0.0,199601172404529,309,0500000US48309


In [12]:
# output the normalized data to a csv file
NOAA_History_Input.to_csv(f'NOAA_FIPS_Normalized_EventHistory_1996_2022.csv', header=True, index=False)
print('end of file combine and save function', len(NOAA_History_Input))

end of file combine and save function 1062498


### end of processing